In [ ]:
import torch
from transformers import (
    AutoTokenizer, AutoModelForCausalLM
)
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

/Users/ethan/Documents/GitHub/BuffettAI/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tokenizer = AutoTokenizer.from_pretrained("oopere-FinChat-XS")
base_model = AutoModelForCausalLM.from_pretrained("oopere/FinChat-XS")
buffetAI = AutoModelForCausalLM.from_pretrained("oopere-FinChat-XS")

In [29]:
df = pd.read_csv('../../data/buffet_qna_finetune_ready.csv')
df = df.dropna()

# Identify rare classes
class_counts = df['Section'].value_counts()
rare_classes = class_counts[class_counts < 11].index
print(rare_classes)

# Combine rare classes into 'Other'
df['Section'] = df['Section'].replace(rare_classes, 'Other')

# Update Section_code after modification
df['Section'] = df['Section'].astype('category')
df['Section_code'] = df['Section'].cat.codes

df['Section'] = df['Section'].astype('category')
df['Section_code'] = df['Section'].cat.codes

# Split the dataset into training, validation, and test sets, proportionately by Sections column
# This will ensure that each split has a similar distribution of the Sections column

train_df, test_df = train_test_split(df, test_size=0.05, stratify=df['Section_code'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['Section_code'], random_state=42)

Index(['The Investment Industry', 'Technology', 'Education', 'Gold'], dtype='object', name='Section')


In [ ]:
def buffett_answer(question, max_new_tokens=256):
    prompt = f"Question: {question}\nAnswer:"
    messages = [
        {"role": "user", "content": prompt}
    ]   
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
    with torch.no_grad():
        output_ids = base_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,  # clearly defined stop token
            pad_token_id=tokenizer.eos_token_id   # to handle padding gracefully
        )
    inputs = tokenizer(prompt, return_tensors="pt").to(buffetAI.device)
    with torch.no_grad():
        buffet_output_ids = buffetAI.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,  # clearly defined stop token
            pad_token_id=tokenizer.eos_token_id   # to handle padding gracefully
        )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True), tokenizer.decode(buffet_output_ids[0], skip_special_tokens=True)

# Try it
test_q = "How should I think about investing during a recession?"
test_q = "Do you have any investing tips?"
base_response, buffet_response = buffett_answer(test_q)
print("-"* 10 + "Normal Response" + "-"*10)
print(base_response)
print("-"* 10 + "BuffetAI Response" + "-"*10)
print(buffet_response)
# "Question: How should I think about investing during a recession? 
#  Answer: I focus on companies with strong fundamentals..."


----------Normal Response----------
system
You are a helpful AI assistant named SmolLM, trained by Hugging Face
user
Question: Do you have any investing tips?
Answer:
assistant
Investment advice can be obtained from various resources. The most common ones include the FINRA Investor Education Foundation and NerdWallet's Investing for Beginners section of their website. Other sources may also provide useful information on investment strategies that could help one achieve financial stability. 

It is important to consult with an expert such as your own financial advisor or someone who has experience in finance before making decisions about investments. It would not only benefit you but others too if everyone had access to reliable sources which advise accordingly without causing unnecessary harm. 
Additionally, there should always consider how much risk one wants to take on when deciding what type of stocks they want to invest in. One must ensure it doesnt compromise future goals based so

## BLEU evaluation

In [ ]:
# Generate predictions
base_preds = []
buffet_preds = []
references = []

for i, row in test_df.iterrows():
    question = row['Questions']
    reference = row['Answers']

    base_resp, buffet_resp = buffett_answer(question)
    
    # Append generated and reference responses
    base_preds.append(base_resp.replace(f"Question: {question}\nAnswer:", "").strip())
    buffet_preds.append(buffet_resp.replace(f"Question: {question}\nAnswer:", "").strip())
    references.append([reference.strip()])  # BLEU expects list of references per prediction

#save the preds
base_preds_df = pd.DataFrame(base_preds, columns=["Base_Predictions"])
buffet_preds_df = pd.DataFrame(buffet_preds, columns=["Buffet_Predictions"])

In [ ]:
import torch
import pandas as pd
import evaluate

# Load BLEU metric
bleu = evaluate.load("bleu")


# Compute BLEU scores
base_bleu = bleu.compute(predictions=base_preds, references=references)
buffet_bleu = bleu.compute(predictions=buffet_preds, references=references)

# Show results
print("Base Model BLEU Score:", base_bleu["bleu"])
print("BuffetAI BLEU Score:", buffet_bleu["bleu"])